# Part 3: Reverse Mode Automatic Differentiation

Dynamic Reverse mode AD can be implemented by declaring a class to represent a value and the child expressions that the value depends on. We've provided the implementation that was shown in the lecture slides as a basis below, but it's missing some parts that will make it useful.

__Tasks:__

- Addition (`__add__`) is incomplete - can you finish it? 
- Can you also implement division (`__truediv__`), subtraction (`__sub__`) and power (`__pow__`)?

In [10]:
import math

class Var:
    def __init__(self, value):
        self.value = value
        self.children = []
        self.grad_value = None

    def grad(self):
        if self.grad_value is None:
            self.grad_value = sum(weight * var.grad()
                                  for weight, var in self.children)
        return self.grad_value
    
    def __str__(self):
        return str(self.value)

    def __mul__(self, other):
        z = Var(self.value * other.value)
        self.children.append((other.value, z))
        other.children.append((self.value, z))
        return z

    def __add__(self, other):
        z = Var(self.value + other.value)
        self.children.append((1.0, z))
        other.children.append((1.0, z))
        return z

    def __sub__(self, other):
        z = Var(self.value - other.value)
        self.children.append((1.0, z))
        other.children.append((-1.0, z))
        return z

    def __truediv__(self, other):
        z = Var(self.value / other.value)
        self.children.append((1 / other.value, z))
        other.children.append((- self.value / (other.value ** 2), z))
        return z

    def __pow__(self, other):
        z = Var(self.value ** other.value)
        self.children.append((other.value * (self.value ** (other.value - 1)), z))
        other.children.append((math.log(self.value) * (self.value ** other.value), z))
        return z

In [11]:
# Tests

Var(1) + Var(1) / Var(1) - Var(1)**Var(1)


## Implementing math functions

Just like when we were looking at Forward Mode AD, we also need to implement some core math functions. Here's the sine function for a `Var`:

In [12]:
def sin(x):
    z = Var(math.sin(x.value))
    x.children.append((math.cos(x.value), z))
    return z

__Task:__ can you implement the _cosine_ (`cos`), _tangent_ (`tan`), and _exponential_ (`exp`) functions in the code block below?

In [13]:
def cos(x):
    z = Var(math.cos(x.value))
    x.children.append((-math.sin(x.value), z))
    return z

def tan(x):
    # YOUR CODE HERE
    z = Var(math.tan(x.value))
    x.children.append((math.tan(x.value) ** 2, z))
    return z

def exp(x):
    z = Var(math.exp(x.value))
    x.children.append((math.exp(x.value), z))
    return z


In [14]:
# Tests
assert cos(Var(0)).value == 1
assert tan(Var(0)).value == 0
assert exp(Var(0)).value == 1


## Time to try it out

We're now in a position to try our implementation.

__Tasks:__ 

- Try running the following code to compute the value of the function $z=x\cdot y+sin(x)$ given $x=0.5$ and $y=4.2$, together with the derivative $\partial z/\partial x$ at that point. 
- Verify that the result is correct by hand-differentiating the function.

In [15]:
x = Var(0.5)
y = Var(4.2)
z = x * y + sin(x)
print('z:', z)

z.grad_value = 1.0 #Note that we have to 'seed' the gradient of z to 1 (e.g. ∂z/∂z=1) before computing grads
print('∂z/∂x:',x.grad())

z: 2.579425538604203
∂z/∂x: 5.077582561890373


__Task:__ Now use the code block below to compute the derivative $\partial z/\partial y$ of the above expression (at the same point $x=0.5, y=4.2$ as above). Store the resultant gradient in the variable `dzdy`. Verify by hand that the result is correct.

In [22]:

dzdy = y.grad()
print('∂z/∂y:', dzdy)

∂z/∂y: 0.5


In [23]:
assert dzdy


## Differentiating Algorithms

Now, let's look at doing something wacky: differentiate an algorithm. For this example, we'll use an algorithm that is in a sense static (in this particular case the upper limit of the for loop is predetermined). However, it is not difficult to see that AD is much more general, and could even be applied to stochastic algorithms (say if we replaced the upper limit of the loop below with `Math.floor(Math.random() * 10)` for example).

__Task:__ Consider the following algorithm and in the box below it manually compute the value of $z$ and the gradient $\partial z/\partial x$ at the end of execution.

In [ ]:
x = Var(0.5)
z = Var(1)
for i in range(0,2):
    z = (z + Var(i)) * x * x

$z = (((1 + 0) * x * x) + 1) * x * x$
$z = (x^2 + 1) * x * x$
$z = x^4 + x^2$
$z = 0.3125$

$\frac{\delta z}{\delta x} = 4x^3 + 2x = 1.5$

__Task__: Now use the code block below to print out the gradient computed by our reverse AD by storing the result in a variable called `grad`. Does it match?

In [28]:
z.grad_value = 1.0
grad = x.grad()
print(grad)

1.5


In [ ]:
# Tests
assert grad


__Task:__ Finally, use the code block below to experiment and test the other math functions and methods you created.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()